In [ ]:
import tensorflow as tf
import sagemaker

In [ ]:
tf.__version__

In [ ]:
import keras

In [ ]:
import boto3
sagemaker_session = sagemaker.Session()

training_input_path = "s3://utec-workshop-cats-n-dogs/data/"
training_input_path

In [ ]:
tf_version = tf.__version__
tf_version

role = sagemaker.get_execution_role()
role

In [ ]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='model.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.g4dn.xlarge',
                          framework_version='2.4', 
                          py_version='py37',
                          script_mode=True,
                          hyperparameters={
                              'epochs': 1,
                          }
                         )

#Training
tf_estimator.fit({'train': training_input_path})

In [ ]:
import time
tf_endpoint_name = 'dog-cat-classifier'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
tf_predictor = tf_estimator.deploy(initial_instance_count=1,instance_type='ml.m5.4xlarge',
                                   endpoint_name=tf_endpoint_name)

In [ ]:
from tensorflow.python.keras.preprocessing.image import load_img
import numpy as np

In [ ]:
img = load_img("./test_images/9718.jpg", target_size=(256, 256))
img = np.array(img).reshape((1, 256, 256, 3))

In [ ]:
img.shape

In [ ]:
predictions = tf_predictor.predict(img)['predictions'] #returns a list of lists

In [ ]:
print("dog" if predictions[0][0]>0 else "Cat")